# Analyse du fonds Daniel Thurre


## 1. Importation des bibliothèques logicielles et des routines

In [ ]:
# Pour Section 3
#%pip install geopandas
#%pip install deep-translator
#%pip install geopy
#%pip install highlight-text

# Pour Section 4
#!pip3 install keras
#!pip3 install tensorflow

# Pour Section 5
#%pip install retina-face

In [ ]:
# gestion des données
import pandas as pd
import geopandas as gpd

from geopy.geocoders import Nominatim

In [ ]:
# production des figures
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from collections import Counter
import matplotlib.ticker as ticker
from matplotlib.ticker import FormatStrFormatter
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import matplotlib.patches as patches
from highlight_text import fig_text
import matplotlib.lines as mlines
import matplotlib.font_manager as font_manager
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

matplotlib.rc('font',family='Times New Roman')

In [ ]:
# traduction 
from deep_translator import GoogleTranslator
from deep_translator import DeeplTranslator

In [ ]:
# partionnement des images

# chargement et analyse des images  
from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array 
from keras.applications.vgg16 import preprocess_input 

# models 
from keras.applications.vgg16 import VGG16 
from keras.models import Model

# partitionnement et reduction dimensionelle
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [ ]:
# reconnaissance faciale 

from retinaface import RetinaFace

In [ ]:
# reste (os, math, date)

import os
import numpy as np
import time
import datetime
today = datetime.date.today()
print("Dernière mise à jour : ", today)

from random import randint
import pickle
import cv2
import requests 

## 2. Importation des données et création du tableur


In [ ]:
os.getcwd()

In [ ]:
# chemin vers le dossier contenant les images
path = '/Users/aureliavalterio/Desktop/Mémoire_HN/OK_VF/'

# lecture du dossier et création de la liste
dossier = os.listdir(path)

# estimation du nombre d'image dans la liste 
# il se peut qu'un ou deux fichiers autres que des images se soient glissés dans la liste
# ces dernier seront mis de côtés lors de l'analyse
print('Le nombre d\'images est',len(dossier),'.')
dossier[:10]

In [ ]:
# création de la liste pour enregistrer le dictionnaire 
liste_dic = []

# sélection des fichiers avec des extensions images, définition des extensions
ext = [".jpeg", ".png", ".jpg"] 

# création du dictionnaire
for files in dossier:
    if files.endswith(tuple(ext)): # sélection des fichiers avec des extensions images
        file, ext = os.path.splitext(files) # séparation des données pour chaque images
        file_info = file.split("_")
#        print(file_info) # Décommenter si message d'erreur pour corriger liste des titres 
        dic = {
            "numéro": file_info[0],
            "type": file_info[1],
            "dénomination": file_info[2],
            "siècle": file_info[3],
            "ville_prod": file_info[4],
            "pays_prod": file_info[5],
            "ville_cons": file_info[6],
            "pays_cons": file_info[7],
            "commentaire": file_info[8],
        }
        liste_dic.append(dic)

# Impression du dictionnaire
print(liste_dic)

In [ ]:
df = pd.DataFrame.from_dict(liste_dic)
df

# 3 Description du corpus de données

### 3.1 Analyse des types présents dans le fonds

In [ ]:
df1 = df[['type']]
df1.head()

In [ ]:
# Répartition des types présents dans le fonds
Type = df1['type'].value_counts()
Type = Type.to_frame().reset_index()
Type

In [ ]:
# Création d'un nouveau tableur Types top + reste
Type_top_plus_rest = df1['type'].value_counts()
Type_top_plus_rest = Type_top_plus_rest.to_frame().reset_index()
Type_top_plus_rest

# Nous gardons les types apparaissant plus de 10 fois dans le fonds, c'est à dire les 8 premiers.
start_row = 8

# Compte de tous les autres types et regroupement dans une nouvelle catégorie 'Autres'.
Type_top_plus_rest.iloc[start_row] = Type_top_plus_rest.iloc[start_row:].sum()
Type_top_plus_rest = Type_top_plus_rest.drop(start_row+1)
Type_top_plus_rest = Type_top_plus_rest.iloc[:start_row+1]


# Nouvelles entêtes des colonnes.
Type_top_plus_rest.columns = ['Type', 'Nombre d\'objets']

# Nommer la nouvelle catégorie 'Autres'.
Type_top_plus_rest.loc[start_row,'Type']='Autres'


Type_top_plus_rest


In [ ]:
# diagramme circulaire

# création de la figure
plt.figure(figsize = (10, 10))

# choix de la donnée à représenter, ici le nombre d'objets
x = Type_top_plus_rest["Nombre d'objets"]

# création du diagramme
plt.pie(x, 
        labels = Type_top_plus_rest["Type"], # choix de l'étiquette, ici les noms des types
#        explode =  (0, 0.2),
        autopct = lambda x: str(round(x, 2)) + r'%', # affichage des %'s arrondis à une décimale
        pctdistance = 0.8, 
        labeldistance = 1.1, # distance entre le diagramme et les étiquettes
        shadow = False, # ombrage oui/non
        colors = ['#d73027','#f46d43','#fdae61','#fee090','#ffffbf','#e0f3f8','#abd9e9','#74add1','#4575b4'], # couleurs
        wedgeprops={"alpha": 0.8}, # opacité
       textprops={'fontsize': 20}, # taille du texte
       startangle=90) # choix de l'angle de départ, ici, on représente la catégorie des reliquaires (~50%) sur la gauche du diagramme
#plt.legend(loc=1,fontsize= 20) # pas de légence
plt.text(-.8, -1.2, 'Auteure :  Valterio Aurelia', ha='right', va='bottom') # pied-de-page 
plt.title("Distribution des types", fontsize=25,y=.95) # Titre

plt.savefig('Figs/Types_camembert.pdf',bbox_inches="tight") # Sauvegarde du diagramme dans un dossier Figs.
plt.show() # Affichage du diagramme

In [ ]:
# diagramme diagramme en barres

# Création du diagramme (type, couleur et opacité des barres, couleur et opacité des contours, taille de la figure)
ax = Type_top_plus_rest.set_index("Type").plot(kind = "bar", figsize=(10,5),
      color=([[0.7, 0, 0, .3]]),edgecolor=([[0.7, 0, 0, 1.]]))

# Étiquettes des axes
ax.set_xlabel("Type", fontsize=15) # étiquette de l'axe des abscisses (x)
ax.set_ylabel('Nombre d\'objets', fontsize=15) # étiquette de l'axe des ordonnées (y)
ax.tick_params(axis='both', which='major', labelsize=12) # choix des ticks

# Titre
ax.set_title("Distribution des types", fontsize=25,y=1.01) 

# Légende 
ax.get_legend().remove() # supression de la légende     
#ax.legend(fontsize=12)

# pied de page
ax.text(0.115, -0.2, 'Auteure :  Valterio Aurelia',transform=plt.gcf().transFigure)

# enregistrement de l'image
plt.savefig('Figs/Types_bars.pdf',bbox_inches="tight")

### 3.2 Analyse chronologiques des objets présents dans le fonds

In [ ]:
df1 = df[['siècle']] # création du nouveau tableur
df1= df1.replace('NaN', np.nan) # remplacement des entrées 'NaN' (string) par np.nan (numpy float)
df1.dropna(inplace=True) # élimination des entrées nan.
df1.head(80)

In [ ]:
df1.siècle.unique()

In [ ]:
# On sépare la colonne siècle avec la commande split et les ":" dans deux nouvelles colonnes siècle_min et siècle_max
df1[['siècle_min', 'siècle_max']] = df1['siècle'].str.rsplit(":",n=1, expand=True)
df1

In [ ]:
# Boucle for pour remplacer les None par les valeurs de la colonne siècle_min.
i=0 # indice de la ligne
col_index = df1.columns.get_loc('siècle_max') # indice de la colonne siècle_max
for item in df1.siècle.str.split(":"): # itération sur toutes les lignes du tableur
    if len(item)==1: # si la longueur de item est 1 (un seul siècle, pas de séparation)
        if item[0]=='NaN':  # on vérifie que l'entrée est bien un siècle et non un NaN
            df1.iloc[i,col_index] = 'NaN'
        else:
            df1.iloc[i,col_index] = str(int(item[0])) # on copie l'entrée dans la colonne siècle_max
    i+=1
    
df1

In [ ]:
# Tableur occurences siècle estimation basse
Siècle_estimation_basse = df1['siècle_min'].value_counts()
Siècle_estimation_basse = Siècle_estimation_basse.to_frame().reset_index()
Siècle_estimation_basse

In [ ]:
# Tableur occurences siècle estimation haute
Siècle_estimation_haute = df1['siècle_max'].value_counts()
Siècle_estimation_haute = Siècle_estimation_haute.to_frame().reset_index()
Siècle_estimation_haute

In [ ]:
# récupération des données en float
s_b = Siècle_estimation_basse['siècle_min'].to_numpy().astype(float) # siècles estimation basse
count_b = Siècle_estimation_basse['count'].to_numpy() # nombre d'occurences siècles estimation basse
s_m = Siècle_estimation_haute['siècle_max'].to_numpy().astype(float) # siècles estimation haute
count_m = Siècle_estimation_haute['count'].to_numpy() # nombre d'occurences siècles estimation haute

# création de la figure
f, ax = plt.subplots(figsize=(10,5),facecolor='w',edgecolor='k')

# largeur des barres
width = 0.3

# diagramme en barres, estimation basse, décalage de width/2 vers la gauche, largeur des barres width 
# couleur des barres et du cadre RGB [0.7, 0, 0] opacité des barres 0.3, opacité des cadres 1.0
ax.bar(s_b-width/2, count_b,width,color=([[0.7, 0, 0, .3]]),edgecolor=([[0.7, 0, 0, 1.]]),label='Estimation basse')

# diagramme en barres, estimation basse, décalage de width/2 vers la droite, largeur des barres width 
# couleur des barres et du cadre RGB [0.17, 0.55, 0.74] opacité des barres 0.3, opacité des cadres 1.0
ax.bar(s_m+width/2, count_m,width,color=([[0.17, 0.55, 0.74, .3]]),edgecolor=([[0.17, 0.55, 0.74, 1.]]),label='Estimation haute')

# Étiquette axe des x
ax.set_xlabel("Siècle", fontsize=15) 
# Étiquette axe des y
ax.set_ylabel("Nombre d'objets", fontsize=15) 
# Ticks
ax.xaxis.set_major_locator(MultipleLocator(1))
ax.tick_params(axis='both', which='major', labelsize=12)

# Titre
ax.set_title("Répartition chronologique", fontsize=25,y=1.01) 

# Légende, loc=2 (en haut à gauche)
ax.legend(fontsize=15,loc=2)

# Pied-de-page en bas à gauche
ax.text(0.115, -0.05, 'Auteure : Valterio Aurelia',transform=plt.gcf().transFigure)

# Sauvegarde de la figure dans le dossier Figs
plt.savefig('Figs/Siècles.pdf',bbox_inches="tight")

### 3.3 Analyse chronologique et par type des objets présents dans le fonds

In [ ]:
# Même commandes que précédemment pour crééer la dataframe avec les siècles min et max
df1 = df[['siècle','type']]
df1= df1.replace('NaN', np.nan)
df1.dropna(inplace=True)
df1[['siècle_min', 'siècle_max']] = df1['siècle'].str.rsplit(":",n=1, expand=True)
i=0
col_index = df1.columns.get_loc('siècle_max')
for item in df1.siècle.str.split(":"):
    if len(item)==1:
        if item[0]=='NaN':
            df1.iloc[i,col_index] = 'NaN'
        else:
            df1.iloc[i,col_index] = str(int(item[0]))
#    if len(item)==3:
#        print(item)
    i+=1
    
df1

In [ ]:
# On crée une nouvelle colonne new_type
df1['new_type'] = df1['type']

# Répartition des types présents dans le fonds afin d'identifier les quatres plus fréquents
Type = df1['type'].value_counts()
Type = Type.to_frame().reset_index()

Type.head(4)

In [ ]:
df1.loc[df1['new_type'].value_counts()[df1['type']].values < Type.iloc[3, 1], 'new_type'] = "Autre"
df1

In [ ]:
df1['new_type'].unique()

In [ ]:
df2 = df1[['siècle_min','new_type']]
df2['count']=1
df2

In [ ]:
df3 = df2.pivot_table(
    index=['siècle_min'],
    columns='new_type',
    values='count',aggfunc='sum').reset_index().fillna(0)
df3['siècle_min'] = df3['siècle_min'].astype(float)
df3.sort_values("siècle_min",inplace=True)
df3

In [ ]:
# Création du diagramme en barres, on choisit stacked=True afin d'avoir la fragmentation par type
# les couleurs pour les 5 catégories sont choisies suivant la convention RBG et l'opacité alpha : [R,G,B,alpha].
ax = df3.sort_values("siècle_min").plot.bar(x='siècle_min', stacked=True,  figsize=(10,5), 
                                            color=([[0.84, 0.1, 0.1, .8],[0.99, 0.68, 0.38, .8],[1, 1, 0.74, .8],
                                                    [0.67, 0.85, 0.91, .8],[0.17, 0.48, 0.71, .8]]))

# Étiquette axe des x
ax.set_xlabel("Siècle", fontsize=15) 
# Étiquette axe des x
ax.set_ylabel("Nombre d'objets", fontsize=15) 


# Ticks
ax.tick_params(axis='both', which='major', labelsize=12)

# Axe des x, on soit faire apparaitre le siècle au format numérique mais sans décimales après la virgule.
labels = [item.get_text() for item in ax.get_xticklabels()]
i=0
for item in labels:
    item = item.split('.')[0]
    labels[i]=item
    i+=1

ax.set_xticklabels(labels)

# Titre du graphique
ax.set_title("Répartition chronologique par type", fontsize=25,y=1.01) 


# Légende     
ax.legend(fontsize=15)

# Pied-de-page    
ax.text(0.1, -0.0, 'Auteure : Valterio Aurelia',transform=plt.gcf().transFigure)

# Sauvegarde dans le dossier Figs
plt.savefig('Figs/Siècles_vs_types.pdf',bbox_inches="tight")

plt.show()


In [ ]:
types_array = np.array([Type.iloc[0,0],Type.iloc[1,0],Type.iloc[2,0],Type.iloc[3,0],'Autre'])
types_array


for i in range(5):
    ax = df3.sort_values("siècle_min").plot.bar(x='siècle_min',y=types_array[i],figsize=(10,5),color=([[0.7, 0, 0, .3]]),edgecolor=([[0.7, 0, 0, 1.]]))

    ax.set_xlabel("Siècle", fontsize=16) #titre de l'axe des abscisses
    ax.set_ylabel("Nombre d'objets", fontsize=16) #titre de l'axe des abscisses

    ax.tick_params(axis='both', which='major', labelsize=12)

    labels = [item.get_text() for item in ax.get_xticklabels()]
    j=0
    for item in labels:
        item = item.split('.')[0]
        labels[j]=item
        j+=1

    ax.set_xticklabels(labels)

    ax.set_title("Répartition chronologique pour le type \""+types_array[i]+"\"", fontsize=20) 
    ax.text(0.1, -0.0, 'Auteure : Valterio Aurelia',transform=plt.gcf().transFigure)

    ax.get_legend().remove()

    plt.savefig('Figs/Siècles_vs_'+types_array[i]+'.pdf',bbox_inches="tight")

    plt.show()


### 3.4 Analyse géographique

#### 3.4.1 Répartition par pays de production

In [ ]:
df1 = df[['pays_prod']]
df1= df1.replace('NaN', np.nan) # Elimination des entrées Nan
df1.dropna(inplace=True)        # Elimination des entrées Nan
df1["pays_prod"] = df1["pays_prod"].str.split(":") # Séparation des entrées avec plusieurs pays de production.
df1 = df1.explode("pays_prod").reset_index(drop=True) # Création de nouvelles lignes avec un pays par ligne.
df1

In [ ]:
Countries = df1['pays_prod'].value_counts()
Countries = Countries.to_frame().reset_index()
Countries

In [ ]:
# Traduction des noms de pays en anglais
Countries['country_prod'] = Countries['pays_prod'] # Copie des noms de pays dans une nouvelle colonne à traduire.

# Boucle de traduction
i=0
col_index = Countries.columns.get_loc('country_prod')
col_index
for item in Countries.pays_prod:
    Countries.iloc[i,col_index] = GoogleTranslator(source='fr', target='en').translate(item) # Traduction de la ligne en englais
    print(item,GoogleTranslator(source='fr', target='en').translate(item) ) # Impression du nom en anglais afin de vérifier la traduction
    i+=1

Countries_en = Countries[['country_prod','count']]
Countries_en

In [ ]:
Countries_en.loc[Countries_en.country_prod=='England', 'country_prod'] = 'United Kingdom'
Countries_en.loc[Countries_en.country_prod=='Scotland', 'country_prod'] = 'United Kingdom'
Countries_en.loc[Countries_en.country_prod=='Sicily', 'country_prod'] = 'Italy'
Countries_en.loc[Countries_en.country_prod=='Swiss', 'country_prod'] = 'Switzerland'
Countries_en.loc[Countries_en.country_prod=='Türkiye', 'country_prod'] = 'Turkey'
Countries_en.loc[Countries_en.country_prod=='Suede', 'country_prod'] = 'Sweden'
Countries_en.loc[Countries_en.country_prod=='Czech Republic', 'country_prod'] = 'Czechia'
Countries_en.loc[Countries_en.country_prod=='Czechian', 'country_prod'] = 'Czechia'
Countries_en.loc[Countries_en.country_prod=='The Netherlands', 'country_prod'] = 'Netherlands'
Countries_en

In [ ]:
Countries_en = Countries_en.groupby(["country_prod"]).agg({"count":"sum"}).reset_index()
Countries_en

In [ ]:
# importation du fichier avec les coordonnées géographiques des pays du monde
world = gpd.read_file("/Users/aureliavalterio/Desktop/Mémoire_HN/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp")

# croisement entre le tableur world et le tableur country_en
data = world.merge(Countries_en, how='left',
                    left_on='NAME', right_on='country_prod')
# supression des pays qui n'ont pas produit d'objets
data.dropna(subset=['count'], inplace=True)

# replacement du nombre d'occurences par un pourcentage
data['count'] = round(data['count']/data['count'].sum()*100,1)

# vérification
data

In [ ]:
# création de la figure
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

# choix des couleurs de colorisation des pays
cmap = cm.Oranges # tons orangés
min_rate, max_rate = -0, 20 # échelle de colorization, dégradé entre 0 et 20%
norm = mcolors.Normalize(vmin=min_rate, vmax=max_rate) # Normalisation de l'échelle


# représentation des données, pays colorisés suivant le nombre d'objets produits
data.plot(column='count', cmap=cmap, norm=norm,
          edgecolor='black', linewidth=0.2,alpha=0.85, ax=ax)

# représentation des frontières en noir (et des pays non producteurs en gris clair)
world.plot(
    ax=ax,
    color=("lightgrey"),
    edgecolor="black",
    alpha=.05
)

# Choix du domaine représenté (ici Europe + Moyen-Orient)
ax.set_xlim(-10, 65)
ax.set_ylim(20, 70)
ax.axis('off')


## Annotation textuelles
# extraction de la position des centres des pays.
data_projected = data.to_crs(epsg=3035)
data_projected['centroid'] = data_projected.geometry.centroid
data['centroid'] = data_projected['centroid'].to_crs(data.crs)
# liste des pays à annoter
countries_to_annotate = ['France', 'Italy', 'Switzerland','Georgia','Austria',
                         'Syria','Sweden','Czechia','Netherlands',
                         'Belgium', 'Germany','United Kingdom','Turkey','Iran','Egypt','Spain'] 

# Ajustement manuel de la position des texte 'Nom du pays':(dx,dy) où dx et dy sont les variation horizontales et verticales de la position
adjustments = {
    'France': (9, 3),
    'Italy': (-3, 1.5),
    'Lithuania': (0, -0.6),
    'Finland': (0, -2.5),
    'Romania': (0, -0.5),
    'Bulgaria': (0, -0.6),
    'Greece': (-1.2, -0.8),
    'Croatia': (0, -1),
    'Cyprus': (0, -1),
    'Ireland': (0, -1),
    'Malta': (0, -1),
    'Slovenia': (0, -1),
    'Slovakia': (-0.7, -0.8),
    'Estonia': (0, -0.7),
    'Latvia': (0, -0.5),
    'Belgium': (0, -0.7),
    'Austria': (1, -1),
    'Spain': (0, -1),
    'Portugal': (-0.5, -1),
    'Luxembourg': (0, -1),
    'Germany': (-0.2, 0),
    'Hungary': (-0.3, -1),
    'Czechia': (0, -1),
    'Poland': (0, -1),
    'Sweden': (-1.5, -1),
    'Denmark': (0, -1),
    'Netherlands': (-1.5,-0.5),
    'United Kingdom': (0, -2),
    'Switzerland': (0, -1),
    'Turkey':(0, -1.),
    'Iran':(0, 0),
    'Egypt':(-1, -1),
    'Georgia':(0, -1),
    'Czechia':(0, -1),
    'Syria':(-0.5, -0.8),
    'Sweden':(-1.5, 0),
}

# annotation
for country in countries_to_annotate:

    # position du texte
    centroid = data.loc[data['NAME'] == country, 'centroid'].values[0]
    x, y = centroid.coords[0]

    # correction de la position
    x += adjustments[country][0]
    y += adjustments[country][1]

    # texte
    code = data.loc[data['NAME'] == country, 'SOV_A3'].values[0]
    rate = data.loc[data['NAME'] == country, 'count'].values[0]
    ax.annotate(f'{code[:2]} {rate}%', (x, y), textcoords="offset points", xytext=(5, 5),
                ha='center', fontsize=10, fontfamily='DejaVu Sans', color='black')


# Titre
ax.set_title("Répartition géographique des lieux d'origine des objets", fontsize=25,y=1.01) 


# pied-de-page
text = "<Auteure>: Aurelia Valterio\n<Source>: https://www.naturalearthdata.com/"
fig_text(0.075, 0.23,
         s=text,
         color='black',
         fontsize=9,
         highlight_textprops=[{"fontweight": 'bold'},
                              {"fontweight": 'bold'}],
         ax=ax)


# création de la figure
plt.tight_layout()
plt.savefig('Figs/Geo_pays_prod_full.pdf',bbox_inches="tight") # sauvegarde dans le dossier Figs
plt.show()

In [ ]:
# création de la figure
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

# choix des couleurs de colorisation des pays
cmap = cm.Oranges # tons orangés
min_rate, max_rate = -0, 30 # échelle de colorization, dégradé entre 0 et 30%
norm = mcolors.Normalize(vmin=min_rate, vmax=max_rate) # Normalisation de l'échelle



# représentation des données, pays colorisés suivant le nombre d'objets produits
data.plot(column='count', cmap=cmap, norm=norm,
          edgecolor='black', linewidth=0.2,alpha=0.85, ax=ax)

# représentation des frontières en noir (et des pays non producteurs en gris clair)
world.plot(
    ax=ax,
    color=("lightgrey"),
    edgecolor="black",
    alpha=.05
)

# Choix du domaine représenté (ici Europe)
ax.set_xlim(-10, 32)
ax.set_ylim(32, 70)
ax.axis('off')



## Annotation textuelles
# extraction de la position des centres des pays
data_projected = data.to_crs(epsg=3035)
data_projected['centroid'] = data_projected.geometry.centroid
data['centroid'] = data_projected['centroid'].to_crs(data.crs)
# liste des pays à annoter
countries_to_annotate = ['France', 'Italy', 'Switzerland','Georgia','Austria',
                         'Syria','Sweden','Czechia','Netherlands',
                         'Belgium', 'Germany','United Kingdom','Turkey','Iran','Egypt','Spain']
adjustments = {
    'France': (10, 3),
    'Italy': (-0.5, -0),
    'Lithuania': (0, -0.6),
    'Finland': (0, -2.5),
    'Romania': (0, -0.5),
    'Bulgaria': (0, -0.6),
    'Greece': (-1.3, -0.),
    'Croatia': (0, -1),
    'Cyprus': (0, -1),
    'Ireland': (0, -1),
    'Malta': (0, -1),
    'Slovenia': (0, -1),
    'Slovakia': (-0.7, -0.8),
    'Estonia': (0, -0.7),
    'Latvia': (0, -0.5),
    'Belgium': (-0.1, -0.5),
    'Austria': (0.2, -0.5),
    'Spain': (0, -1),
    'Portugal': (-0.5, -1),
    'Luxembourg': (0, -1),
    'Germany': (-0.2, 0),
    'Hungary': (-0.3, -1),
    'Czechia': (-0.5, -0.5),
    'Poland': (0, -1),
    'Sweden': (-1.5, -1),
    'Denmark': (-0.8, -0.1),
    'Netherlands': (-0.2, -0.3),
    'United Kingdom': (1, -2),
    'Switzerland': (-0.2, -0.5),
    'Turkey':(-6,-1.),
    'Iran':(0, 0),
    'Egypt':(0, 0),
    'United States of America':(0, 0),
    'Sweden':(-1, 0),
    'Georgia':(0, 0),
    'Egypt':(-1, -1),
    'Syria':(-0.5, -0.8),
}


# annotation
for country in countries_to_annotate:

    # position du texte
    centroid = data.loc[data['NAME'] == country, 'centroid'].values[0]
    x, y = centroid.coords[0]

    # correction de la position
    x += adjustments[country][0]
    y += adjustments[country][1]

    # texte
    code = data.loc[data['NAME'] == country, 'SOV_A3'].values[0]
    rate = data.loc[data['NAME'] == country, 'count'].values[0]
    ax.annotate(f'{code[:2]} {rate}%', (x, y), textcoords="offset points", xytext=(5, 5),
                ha='center', fontsize=10, fontfamily='DejaVu Sans', color='black')

# Titre
ax.set_title("Répartition géographique des lieux d'origine des objets", fontsize=25,y=1.01) 

# Annotation des pays extra-européens
ax.annotate('Hors-carte :', (-5, 65), textcoords="offset points", xytext=(5, 5),
                ha='center', fontsize=15,fontweight='bold', color='black')
rateEgypt = data.loc[data['NAME'] == 'Egypt', 'count'].values[0]
ax.annotate('Égypte %1.1f' %rateEgypt+'%', (-5, 64), textcoords="offset points", xytext=(5, 5),
                ha='center', fontsize=15, color='black')

rateGeorgia = data.loc[data['NAME'] == 'Georgia', 'count'].values[0]
ax.annotate('Géorgie %1.1f' %rateGeorgia+'%', (-5, 63), textcoords="offset points", xytext=(5, 5),
                ha='center', fontsize=15, color='black')


rateSyria = data.loc[data['NAME'] == 'Syria', 'count'].values[0]
ax.annotate('Syrie %1.1f' %rateSyria+'%', (-5, 62), textcoords="offset points", xytext=(5, 5),
                ha='center', fontsize=15, color='black')
rateIran = data.loc[data['NAME'] == 'Iran', 'count'].values[0]
ax.annotate('Iran %1.1f' %rateIran+'%', (-5, 61), textcoords="offset points", xytext=(5, 5),
                ha='center', fontsize=15, color='black')


# pied-de-page
text = "<Auteure>: Aurelia Valterio\n<Source>: https://www.naturalearthdata.com/"
fig_text(0.085, .1,
         s=text,
         color='black',
         fontsize=9,
         highlight_textprops=[{"fontweight": 'bold'},
                              {"fontweight": 'bold'}],
         ax=ax)



# création de la figure
plt.tight_layout()
plt.savefig('Figs/Geo_pays_prod.pdf',bbox_inches="tight") # sauvegarde dans le dossier Figs
plt.show()

#### 3.4.2 Répartition par ville de production

In [ ]:
df1 = df[['ville_prod']]
df1= df1.replace('NaN', np.nan) # Elimination des NaN
df1.dropna(inplace=True)        # Elimination des NaN
df1["ville_prod"] = df1["ville_prod"].str.split(":")  # Séparation des entréees qui contiennent plusieurs villes de production.
df1 = df1.explode("ville_prod").reset_index(drop=True) # Éclatement du tableur afin d'avoir une ville par ligne
df1

In [ ]:
Villes_prod = df1['ville_prod'].value_counts()
Villes_prod = Villes_prod.to_frame().reset_index()
Villes_prod.tail(50)

In [ ]:
# Affiner à la main la dataframe
Villes_prod.loc[Villes_prod.ville_prod=='Saint Maurice', 'ville_prod'] = 'Saint Maurice Suisse'
Villes_prod.loc[Villes_prod.ville_prod=='Cologne', 'ville_prod'] = 'Cologne Allemagne'
Villes_prod.loc[Villes_prod.ville_prod=='Bâle', 'ville_prod'] = 'Bâle Suisse'
Villes_prod.loc[Villes_prod.ville_prod=='Trèves', 'ville_prod'] = 'Trèves Allemagne'
Villes_prod.head(50)

In [ ]:
# Initialisation de l'API Nominatim
geolocator = Nominatim(user_agent="MyApp")

location = geolocator.geocode("Aywiers")

print("The latitude of the location is: ", location.latitude)
print("The longitude of the location is: ", location.longitude)

In [ ]:
# Initialisation de l'API Nominatim
geolocator = Nominatim(user_agent="MyApp")

# Définition d'une fonction qui calcule les coordonnées (latitude et longitude) d'une ville
def coordinates(city):
    location = geolocator.geocode(city)
    latitude = 'NaN'
    longitude = 'NaN'
    if location !=None:
        latitude=location.latitude
        longitude=location.longitude
    return latitude, longitude

# Création de deux colonnes vides (latitude et longitude)
Villes_prod[['latitude','longitude']] = ['NaN','NaN']

# Boucle for pour entrer les latitudes et les longitudes des villes avec la fonction coordinates
i=0
col_index_lat = Villes_prod.columns.get_loc('latitude') # indice de la colonne latitude 
col_index_long = Villes_prod.columns.get_loc('longitude') # indice de la colonne longitude 
for item in Villes_prod.ville_prod:
    print(item) # impression du nom de la ville
    location = coordinates(item)  # calcul des coordonnées avec la fonction coordinates
    Villes_prod.iloc[i,col_index_lat]=location[0] # enregistrement de la latitude
    Villes_prod.iloc[i,col_index_long]=location[1]  # enregistrement de la longitude
    time.sleep(1.1) # pause d'1.1 second, car l'API est limité à 1 requête par seconde.
    i+=1

Villes_prod= Villes_prod.replace('NaN', np.nan) # Elimination des NaN
Villes_prod.dropna(inplace=True)                # Elimination des NaN
Villes_prod

In [ ]:
Villes_prod.head(50)

In [ ]:
coordinates('Cologne Allemagne')

In [ ]:
# importer les données géographiques du monde
world = gpd.read_file("/Users/aureliavalterio/Desktop/Mémoire_HN/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp")

# Création de la figure
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

# Création de l'arrière plan : les pays colorisés légèrement
world.plot(ax=ax,color='#fdcc8a',alpha=0.2,edgecolor='None')

# Représentation des données: disque coloré sur chaque ville productrice, la taille dépend du nombre d'objets
sc1 = ax.scatter(x=Villes_prod['longitude'], y=Villes_prod['latitude'],
           s=(Villes_prod['count']/Villes_prod['count'].sum())*2000,
          facecolors='#b30000',alpha=0.3, edgecolors='#b30000')
# Représentation des données: cercle coloré sur chaque ville productrice, la taille dépend du nombre d'objets
sc1 = ax.scatter(x=Villes_prod['longitude'], y=Villes_prod['latitude'],
           s=(Villes_prod['count']/Villes_prod['count'].sum())*2000,
          facecolors='None',alpha=1, edgecolors='#b30000')

# Avant plan : les frontières colorisés légèrement
world.plot(ax=ax,color='None',alpha=.05,edgecolor='black')

# Choix du domaine représenté (ici Europe, même limites que précédemment)
ax.set_xlim(-10, 32)
ax.set_ylim(32, 70)
ax.axis('off')


## Création de la légende
# marqueur pour 1 objet
one = mlines.Line2D([], [], marker='o', linestyle='None',color='#b30000',
                          markersize=(1/Villes_prod['count'].sum())*600,markerfacecolor=([0.70,0,0,0.5]),markeredgecolor=([0.70,0,0,1]), label='1')
# marqueur pour 5 objets
five = mlines.Line2D([], [], marker='o', linestyle='None',color='#b30000',
                          markersize=(5/Villes_prod['count'].sum())*200,markerfacecolor=([0.70,0,0,0.5]),markeredgecolor=([0.70,0,0,1]), label='5')
# marqueur pour 10 objets
ten = mlines.Line2D([], [], marker='o', linestyle='None',color='#b30000',
                          markersize=(10/Villes_prod['count'].sum())*200,markerfacecolor=([0.70,0,0,0.5]),markeredgecolor=([0.70,0,0,1]), label='10')
# marqueur pour 20 objets
twenty = mlines.Line2D([], [], marker='o', linestyle='None',color='#b30000',
                          markersize=(20/Villes_prod['count'].sum())*140,markerfacecolor=([0.70,0,0,0.5]),markeredgecolor=([0.70,0,0,1]), label='20')
# affichage de la légende
plt.legend(handles=[one, five, ten,twenty],loc=2,fontsize=20)

# Titre
ax.set_title("Répartition géographique des villes de production", fontsize=25,y=1.01) 

# pied-de-page
text = "<Auteure>: Aurelia Valterio\n<Source>: https://www.naturalearthdata.com/"
fig_text(0.085, .1,
         s=text,
         color='black',
         fontsize=9,
         highlight_textprops=[{"fontweight": 'bold'},
                              {"fontweight": 'bold'}],
         ax=ax)

# affichage et sauvegarde de la figure
plt.tight_layout()
plt.savefig('Figs/Geo_villes_origine.pdf',bbox_inches="tight")
plt.show()

#### 3.4.3 Répartition par pays de conservation

In [ ]:
df1 = df[['pays_cons']]
df1= df1.replace('NaN', np.nan)
df1.dropna(inplace=True)
df1["pays_cons"] = df1["pays_cons"].str.split(":")
df1 = df1.explode("pays_cons").reset_index(drop=True)
df1

In [ ]:
# Répartition des types présents dans le fonds
Countries = df1['pays_cons'].value_counts()
Countries = Countries.to_frame().reset_index()
Countries

In [ ]:
Countries['country_cons'] = Countries['pays_cons']
i=0
col_index = Countries.columns.get_loc('country_cons')
col_index
for item in Countries.pays_cons:
    Countries.iloc[i,col_index] = GoogleTranslator(source='fr', target='en').translate(item) 
    print(item,GoogleTranslator(source='fr', target='en').translate(item) )
    i+=1

Countries_en = Countries[['country_cons','count']]
Countries_en

In [ ]:
Countries_en.loc[Countries_en.country_cons=='Suede', 'country_cons'] = 'Sweden'
Countries_en.loc[Countries_en.country_cons=='England', 'country_cons'] = 'United Kingdom'
Countries_en.loc[Countries_en.country_cons=='Scotland', 'country_cons'] = 'United Kingdom'
Countries_en.loc[Countries_en.country_cons=='Swiss', 'country_cons'] = 'Switzerland'
Countries_en.loc[Countries_en.country_cons=='USA', 'country_cons'] = 'United States of America'
Countries_en.loc[Countries_en.country_cons=='The Netherlands', 'country_cons'] = 'Netherlands'
Countries_en.loc[Countries_en.country_cons=='Czech Republic', 'country_cons'] = 'Czechia'
Countries_en.loc[Countries_en.country_cons=='Czechian', 'country_cons'] = 'Czechia'
Countries_en.loc[Countries_en.country_cons=='Suede', 'country_cons'] = 'Sweden'


Countries_en = Countries_en.groupby(["country_cons"]).agg({"count":"sum"}).reset_index()
Countries_en

In [ ]:
world = gpd.read_file("/Users/aureliavalterio/Desktop/Mémoire_HN/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp")

data = world.merge(Countries_en, how='left',
                    left_on='NAME', right_on='country_cons')
data.dropna(subset=['count'], inplace=True)
data['count'] = round(data['count']/data['count'].sum()*100,1)
data

In [ ]:
# création de la figure
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

# choix des couleurs
cmap = cm.Oranges # tons orangés
min_rate, max_rate = -0, 30 # échelle de colorization, dégradé entre 0 et 30%
norm = mcolors.Normalize(vmin=min_rate, vmax=max_rate) # Normalisation de l'échelle


# représentation des données, pays colorisés suivant le nombre d'objets conserrvés
data.plot(column='count', cmap=cmap, norm=norm,
          edgecolor='black', linewidth=0.2,alpha=0.85, ax=ax)

# représentation des frontières en noir (et des pays non producteurs en gris clair)
world.plot(
    ax=ax,
    color=("lightgrey"),
    edgecolor="black",
    alpha=.05
)

# choix du domaine représenté (ici Europe uniquement)
ax.set_xlim(-10, 32)
ax.set_ylim(32, 70)
ax.axis('off')




## Annotation textuelles
# extraction de la position des centres des pays
data_projected = data.to_crs(epsg=3035)
data_projected['centroid'] = data_projected.geometry.centroid
data['centroid'] = data_projected['centroid'].to_crs(data.crs)
# liste des pays à annoter
countries_to_annotate = ['France', 'Italy', 'Switzerland','Poland','Sweden','Denmark','Czechia',
                         'Spain','Netherlands','Belgium', 'Germany','United Kingdom',
                         'United States of America','Greece','Sweden','Austria','Georgia']

# Ajustement manuel de la position des texte 'Nom du pays':(dx,dy) où dx et dy sont les variation horizontales et verticales de la position
adjustments = {
    'France': (10, 3),
    'Italy': (-0.5, -0),
    'Lithuania': (0, -0.6),
    'Finland': (0, -2.5),
    'Romania': (0, -0.5),
    'Bulgaria': (0, -0.6),
    'Greece': (-1.3, -0.),
    'Croatia': (0, -1),
    'Cyprus': (0, -1),
    'Ireland': (0, -1),
    'Malta': (0, -1),
    'Slovenia': (0, -1),
    'Slovakia': (-0.7, -0.8),
    'Estonia': (0, -0.7),
    'Latvia': (0, -0.5),
    'Belgium': (-0.1, -0.5),
    'Austria': (0.2, -0.5),
    'Spain': (0, -1),
    'Portugal': (-0.5, -1),
    'Luxembourg': (0, -1),
    'Germany': (-0.2, 0),
    'Hungary': (-0.3, -1),
    'Czechia': (-0.5, -0.5),
    'Poland': (0, -1),
    'Sweden': (-1.5, -1),
    'Denmark': (-0.8, -0.1),
    'Netherlands': (-0.2, -0.3),
    'United Kingdom': (1, -2),
    'Switzerland': (-0.2, -0.5),
    'Turkey':(0, -1.),
    'Iran':(0, 0),
    'Egypt':(0, 0),
    'United States of America':(0, 0),
    'Sweden':(-1, 0),
    'Georgia':(0, 0),
}

# annotation
for country in countries_to_annotate:

    # position du texte
    centroid = data.loc[data['NAME'] == country, 'centroid'].values[0]
    x, y = centroid.coords[0]

    # corrections de la position
    x += adjustments[country][0]
    y += adjustments[country][1]

    # texte
    code = data.loc[data['NAME'] == country, 'SOV_A3'].values[0]
    rate = data.loc[data['NAME'] == country, 'count'].values[0]
    ax.annotate(f'{code[:2]} {rate}%', (x, y), textcoords="offset points", xytext=(5, 5),
                ha='center', fontsize=10, color='black')

# Annotation des pays extra-européens
ax.annotate('Hors-carte :', (-5, 65), textcoords="offset points", xytext=(5, 5),
                ha='center', fontsize=15,fontweight='bold', color='black')
rateUSA = data.loc[data['NAME'] == 'United States of America', 'count'].values[0]
ax.annotate('USA %1.1f' %rateUSA+'%', (-5, 64), textcoords="offset points", xytext=(5, 5),
                ha='center', fontsize=15, color='black')

rateGeorgia = data.loc[data['NAME'] == 'Georgia', 'count'].values[0]
ax.annotate('Géorgie %1.1f' %rateGeorgia+'%', (-5, 63), textcoords="offset points", xytext=(5, 5),
                ha='center', fontsize=15, color='black')






# pied-de-page
text = "<Auteure>: Aurelia Valterio\n<Source>: https://www.naturalearthdata.com/"
fig_text(0.085, .1,
         s=text,
         color='black',
         fontsize=9,
         highlight_textprops=[{"fontweight": 'bold'},
                              {"fontweight": 'bold'}],
         ax=ax)

# titre
ax.set_title("Répartition géographique des lieux de conservation des objets", fontsize=25,y=1.01) 



# sauvegarde de la figure
plt.tight_layout()
plt.savefig('Figs/Geo_pays_cons.pdf',bbox_inches="tight")
# affichage de la figure
plt.show()

## 4 Partitionnement non-supervisé d'un corpus d'images médiévales.

In [ ]:
# chemin vers le dossier contenant les images
path = '/Users/aureliavalterio/Desktop/Mémoire_HN/OK_VF/'

# déplacement du dossier de travail dans le dossier des images
os.chdir(path)


# lecture du dossier et création de la liste
images_to_cluster = os.listdir(path)

# estimation du nombre d'image dans la liste 
# il se peut qu'un ou deux fichiers autres que des images se soient glissés dans la liste
# ces dernier seront mis de côtés lors de l'analyse
images_to_cluster[:10]

In [ ]:
# Chargement du modèle (VGG16) et définition d'une fonction pour extraire les charactéristiques des images avec keras

model = VGG16()
model = Model(inputs = model.inputs, outputs = model.layers[-2].output)

def extract_features(file, model):
    # chargement de l'image comme une matrice 224x224
    img = load_img(file, target_size=(224,224))
    # converstion de 'PIL.Image.Image' en un numpy array
    img = np.array(img) 
    # redimensionnement des données pour le model reshape(num_of_samples, dim 1, dim 2, channels)
    reshaped_img = img.reshape(1,224,224,3) 
    # préparation des images pour le modèle
    imgx = preprocess_input(reshaped_img)
    # création du vecteur avec les charactéristiques
    features = model.predict(imgx)
    return features

# Test de la fonction extract_features

extract_features(images_to_cluster[0], model)

In [ ]:
data = {}
p = '/Users/aureliavalterio/Desktop/Mémoire_HN/images_features.pkl'



# boucle sur toutes les images du corpus
for imgtc in images_to_cluster:
    # extraction des characteristiques et mise à jour du dictionnaire
    try:
        feat = extract_features(imgtc,model)
        data[imgtc] = feat
    # si erreur, sauvegarde des characteristiques dans un fichier pickle (optionel)
    except:
        with open(p,'wb') as file:
            pickle.dump(data,file)

# création d'une liste avec les nom des fichiers
filenames = np.array(list(data.keys()))

# création d'une liste avec les charactéristiques
feat = np.array(list(data.values()))
print(feat.shape)


# redimensionnement pour avoir xxx échantillons avec 4096 vecteurs
feat = feat.reshape(-1,4096)
print(feat.shape)

In [ ]:
# Réduction dimensionelle et partitionnement avec kmeans

pca = PCA(n_components=100, random_state=22)
pca.fit(feat)
x = pca.transform(feat)

# Choix du nombre de cluster
n_clusters=5

kmeans = KMeans(n_clusters, random_state=22)
kmeans.fit(x)

# Impression du nombre de cluster pour chaque image du corpus
print(kmeans.labels_)
print(len(kmeans.labels_))

In [ ]:
# partitionnement des images dans les clusters et définition d'une fonction permettant de voir les images dans chaque cluster

groups = {}
for file, cluster in zip(filenames,kmeans.labels_):
    if cluster not in groups.keys():
        groups[cluster] = []
        groups[cluster].append(file)
    else:
        groups[cluster].append(file)
        
def view_cluster(cluster):
    plt.figure(figsize = (25,25));
    # liste des noms de fichiers dans le cluster 
    files = groups[cluster]
    # limite de 100 images affichées
    if len(files) > 26:
        print(f"Clipping cluster size from {len(files)} to 25")
        files = files[:25]
    # affichage des images présentes dans le cluster
    for index, file in enumerate(files):
        plt.subplot(5,5,index+1);
        img = load_img(file)
        img = np.array(img)
        plt.imshow(img)
        plt.axis('off')
    plt.savefig('/Users/aureliavalterio/Desktop/Mémoire_HN/Figs/Cluster_'+str(cluster)+'.pdf',bbox_inches="tight")

        
for i in range(n_clusters):
    print('Cluster ',i+1)
    view_cluster(i)
    plt.show()

## 5 Reconnaissance faciale et partitionnement des visages

### 5.1 Reconnaissance faciale

In [ ]:
# haar cascade
path = '/Users/aureliavalterio/Desktop/Mémoire_HN/OK_VF/'
name = '394_Reliquaire_Buste du pape Léon 3_19_Aix la Chapelle_Allemagne_Aix la Chapelle_Allemagne_NaN.jpg'
#image_path = path + '261_Reliquaire_NaN_12_NaN_NaN_Paris_France_NaN.jpg'
#image_path = path + '14_Reliquaire_Althée_9_NaN_Allemagne_Sion_Suisse_NaN.jpg'
image_path = path+name


img = cv2.imread(image_path)
gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

face_classifier = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

face = face_classifier.detectMultiScale(
    gray_image, scaleFactor=1.1, minNeighbors=8, minSize=(40, 40))



index=1
plt.subplot(1,len(face)+1,index);

for (column, row, width, height) in face:
    image = cv2.rectangle(img,(column, row),(column + width, row + height),(0, 255, 0),40)
plt.imshow(image)
plt.axis("off")


# Pour afficher
for (column, row, width, height) in face:
    plt.subplot(1,len(face)+1,index+1);
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_rec = image[row:row+width, column:column+width]
#    print(image_rec)
    plt.imshow(image_rec)
    plt.axis("off")
    index+=1
    
plt.show()

In [ ]:
path = '/Users/aureliavalterio/Desktop/Mémoire_HN/OK_VF/'
name = '394_Reliquaire_Buste du pape Léon 3_19_Aix la Chapelle_Allemagne_Aix la Chapelle_Allemagne_NaN.jpg'
#image_path = path + '261_Reliquaire_NaN_12_NaN_NaN_Paris_France_NaN.jpg'
#image_path = path + '14_Reliquaire_Althée_9_NaN_Allemagne_Sion_Suisse_NaN.jpg'
image_path = path+name


faces = RetinaFace.extract_faces(image_path,
                                 threshold=0.8,align = True)

if len(faces)>0:
    fig, ax = plt.subplots(1,len(faces)+1,figsize=(15,10))
    image = mpimg.imread(image_path)
    ax[0].imshow(image)
    ax[0].set_axis_off()   
    
    index=1
    for face in faces:
        ax[index].imshow(face)
        ax[index].set_axis_off()
        index+=1
    plt.show()
    index=1


In [ ]:
path = '/Users/aureliavalterio/Desktop/Mémoire_HN/OK_VF/'


# Pour éviter les doublons : supprimer le dossier s'il existe
newpath = '/Users/aureliavalterio/Desktop/Mémoire_HN/Detected_faces/'
if not os.path.exists(newpath):
    os.makedirs(newpath)

dossier = os.listdir(path)
ext = [".jpeg", ".png", ".jpg"]

count=0
for files in dossier:
    if files.endswith(tuple(ext)):
        try:
            count+=1
            image_path = path+files
            faces = RetinaFace.extract_faces(image_path,threshold=0.8,align = True)
            if len(faces)>0:
                print(count,files)
                fig, ax = plt.subplots(1,len(faces)+1)
                image = mpimg.imread(image_path)
                ax[0].imshow(image)
                ax[0].set_axis_off()   
                index=1
                for face in faces:
                    ax[index].imshow(face)
                    ax[index].set_axis_off()
                    index+=1
            plt.show()
            index=1
            for face in faces:
                fig, ax = plt.subplots()
                ax.imshow(face)
                ax.set_axis_off()
                plt.savefig(newpath+files.split('.')[0]+'_'+str(index)+'.jpg',bbox_inches='tight')
                plt.close(fig)
                index+=1
        except:
            pass

### 5.2 Clustering des visages

In [ ]:
# pour le partionnement des images
# chargement et analyse des images  
from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array 
from keras.applications.vgg16 import preprocess_input 

# models 
from keras.applications.vgg16 import VGG16 
from keras.models import Model

# partitionnement et reduction dimensionelle
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA


In [ ]:
# chemin vers le dossier contenant les images
path = '/Users/aureliavalterio/Desktop/Mémoire_HN/Detected_faces/'

# changement du dossier de travail dans le dossier où se trouvent les images
os.chdir(path)

# lecture du dossier et création de la liste
images_to_cluster = os.listdir(path)

# estimation du nombre d'image dans la liste 
# il se peut qu'un ou deux fichiers autres que des images se soient glissés dans la liste
# ces dernier seront mis de côtés lors de l'analyse

print(images_to_cluster[:10])
print('Le nombre d\'images est',len(images_to_cluster),'.')

In [ ]:
# Chargement du modèle (VGG16) et définition d'une fonction pour extraire les charactéristiques des images avec keras

model = VGG16()
model = Model(inputs = model.inputs, outputs = model.layers[-2].output)

def extract_features(file, model):
    # chargement de l'image comme une matrice 224x224
    img = load_img(file, target_size=(224,224))
    # converstion de 'PIL.Image.Image' en un numpy array
    img = np.array(img) 
    # redimensionnement des données pour le model reshape(num_of_samples, dim 1, dim 2, channels)
    reshaped_img = img.reshape(1,224,224,3) 
    # préparation des images pour le modèle
    imgx = preprocess_input(reshaped_img)
    # création du vecteur avec les charactéristiques
    features = model.predict(imgx)
    return features

# Test de la fonction extract_features

extract_features(images_to_cluster[0], model)

In [ ]:
data = {}
p = '/Users/aureliavalterio/Desktop/Mémoire_HN/images_features.pkl'



# lop through each image in the dataset
for imgtc in images_to_cluster:
    # try to extract the features and update the dictionary
    try:
        feat = extract_features(imgtc,model)
        data[imgtc] = feat
    # if something fails, save the extracted features as a pickle file (optional)
    except:
        with open(p,'wb') as file:
            pickle.dump(data,file)
          
 
# get a list of the filenames
filenames = np.array(list(data.keys()))

# get a list of just the features
feat = np.array(list(data.values()))
print(feat.shape)


# reshape so that there are xxx samples of 4096 vectors
feat = feat.reshape(-1,4096)
print(feat.shape)

In [ ]:
# Dimensional reduction and clustering using kmeans

pca = PCA(n_components=100, random_state=22)
pca.fit(feat)
x = pca.transform(feat)

# ENTER HERE THE NUMBER OF CLUSTERS
n_clusters=6

kmeans = KMeans(n_clusters, random_state=22)
kmeans.fit(x)

# Print the cluster label of each image
print(kmeans.labels_)
print(len(kmeans.labels_))

In [ ]:
# partitionnement des images dans les clusters et définition d'une fonction permettant de voir les images dans chaque cluster

groups = {}
for file, cluster in zip(filenames,kmeans.labels_):
    if cluster not in groups.keys():
        groups[cluster] = []
        groups[cluster].append(file)
    else:
        groups[cluster].append(file)
        
def view_cluster(cluster):
    plt.figure(figsize = (25,25));
    # liste des noms de fichiers dans le cluster 
    files = groups[cluster]
    # limite de 100 images affichées
    if len(files) > 25:
        print(f"Clipping cluster size from {len(files)} to 25")
        files = files[:25]
    # affichage des images présentes dans le cluster
    for index, file in enumerate(files):
        plt.subplot(5,5,index+1);
        img = load_img(file)
        img = np.array(img)
        plt.imshow(img)
        plt.axis('off')
    plt.savefig('/Users/aureliavalterio/Desktop/Mémoire_HN/Figs/Faces_Cluster_'+str(cluster)+'.pdf',bbox_inches="tight")

        
for i in range(n_clusters):
    print('Cluster ',i+1)
    view_cluster(i)
    plt.show()